In [302]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 5 - Fun with document vectors
</center>
</h1>
<div class=h1_cell>
<p>
I want to continue to explore using the values in bag-of-words to build vectors. The general idea is that we will generate a vector for EAP, a vector for HPL and a vector for MWS. How do we get these vectors? Simple. We take a column from bag-of-words. Before going further, let's read in bag-of-words fromn week 4.
</div>

In [303]:
#I am using dropbox so got the url to my file. If you have on local drive, then use file reading code

import json
bag_of_words = json.load(open("bag_of_words.txt"))
sorted_items = sorted(bag_of_words.items())  # need to sort to make sure vectors align
sorted_items[:10]

[(u'aaem', [1, 0, 0]),
 (u'ab', [1, 0, 0]),
 (u'aback', [2, 0, 0]),
 (u'abaft', [0, 0, 1]),
 (u'abandon', [7, 3, 1]),
 (u'abandoned', [11, 13, 5]),
 (u'abandoning', [2, 1, 0]),
 (u'abandonment', [2, 0, 3]),
 (u'abaout', [0, 24, 0]),
 (u'abased', [1, 0, 0])]

<h2>
Challenge 1
</h2>
<div class=h1_cell>
<p>
Let's write a better version of sentence_wrangler. What I noticed this week when going through new books is that I was letting some strange words through. For instance, my sentence_wrangler from last week lets numbers through. And it also lets byte codes through. I think a better design would be to switch from a blacklist (define chars don't want) to a whitelist (define chars that are ok). Change the 3rd argument to the set of legal characters you allow.
<p>
If you want to be fancy, be my guest. Use the 3rd argument to pass in an re pattern that needs to match against each word. Much more elegant.
</div>

In [304]:
import re
def sentence_wrangler(sentence, swords, legal_chars):
    tokes = word_punct_tokenizer.tokenize(sentence)
    result = []
    removed = []
    check = 0
    for word in tokes:
        word = word.replace("_", "")
        if word.lower() not in swords:
            for i in word.lower():
                if i in legal_chars:
                    check = 1
                else: 
                    check = 0
            if check == 1:
                result.append(word.lower())
            else:
                removed.append(word.lower())
        else:
            removed.append(word.lower())
    return (result, removed)

In [305]:
#Here is my whitelist - re pattern would be better. Extra credit if you do it

#legals = r'...'

legals = 'abcdefghijklmnopqrstuvwxyz'

<div class=h1_cell>
<p>
Some other odds and ends. We will need cosine_similarity from prior module, stop words and tokenizer.
<p>
</div>

In [306]:
def cosine_similarity(v1,v2):
    AB = 0.0
    A = 0.0
    B = 0.0
    for i in range(len(v1)):
        AB += (v1[i] * v2[i])
        A += (v1[i]**2)
        B += (v2[i]**2)  
    return AB/((A*B)**(0.5))

In [307]:
from nltk.corpus import stopwords
swords = stopwords.words('english')

In [308]:
from nltk.tokenize import WordPunctTokenizer
word_punct_tokenizer = WordPunctTokenizer()      

<h2>
Ok, let's get to it
</h2>
<div class=h1_cell>
<p>
What I want to know is how "close" 2 books are to each other. I'll build a word-count vector for each book. And then take the cosine similarity. I'll give you a start.
</div>

In [325]:
#item in sorted_items: (word, (eap_val, hpl_val, mws_val))
sorted_items = []

for word in sorted(bag_of_words):
    temp = []
    temp.append(word)
    temp.append(tuple(bag_of_words[word]))
    sorted_items.append(tuple(temp))  

eap_vector = [pair[1][0] for pair in sorted_items]
hpl_vector = [pair[1][1] for pair in sorted_items]
mws_vector = [pair[1][2] for pair in sorted_items]

In [326]:
eap_hpl = cosine_similarity(eap_vector, hpl_vector)
eap_hpl

0.7487332567707315

<h2>
Is that close?
</h2>
<div class=h1_cell>
<p>
The range of the cosine similarity for us is 0..1. Does that make .75 high? It is hard to answer this without having the values for other book combinations. I would say it is high enough to warrant a further look if I was searching for plagiarism. Let's check out some other combos.
</div>

<h2>
Challenge 2
</h2>
<div class=h1_cell>
<p>
Go ahead and do the other 2 comparisons.
</div>

In [327]:
eap_mws = cosine_similarity(eap_vector, mws_vector)
eap_mws

0.7462905158479859

In [328]:
mws_hpl = cosine_similarity(mws_vector, hpl_vector)
mws_hpl

0.7371094246610521

<h2>
Kind of interesting
</h2>
<div class=h1_cell>
<p>
All 3 have roughly same similarity score. I would expect that given they are all gothic novels. Do you think we are catching the gothic/horror genre in our vectors through use of words?
</div>

<h2>
Challenge 3
</h2>
<div class=h1_cell>
<p>
Let's test our conjecture that we are capturing something about gothicness. Let's compare the 3 against Huckleberry Finn by Mark Twain. My gut feeling is that this should not be high on gothic scale. Your goal is to build a huck_vector that you can compare against our existing vectors. Here is what you need to do:
<p>
<ol>
<li>Initialize a huck_dict that has same keys as bag_of_words and each key's value is a count of that word in the Huck Finn book.
<li>Find an online version of Huck Finn. Hint: Project Gutenberg is a great source.
<li>Figure out how to read the book in and to break the book into sentences.
<li>Pass each sentence through sentence_wrangler to get words.
<li>For each word, increase the count for huck_dict[word], but only if word is in bag_of_words. If the word is not in bag_of_words, add it to the list huck_left_out.
</ol>
<p>
Check your results against mine.
</div>

In [341]:
import urllib2
url = "http://www.gutenberg.org/files/76/76-0.txt"
f = urllib2.urlopen(url)


alltext = ""

#count = 0
for line in f.readlines():
    #count += 1
    #if count > 23 and count < 11976:
    alltext += line

allsentences = word_punct_tokenizer.tokenize(alltext.decode('utf-8'))

all_huck_words = bag_of_words.copy()
huck_left_out = []
for word in all_huck_words:
    all_huck_words[word] = 0
for sentence in allsentences:
    wrangled = sentence_wrangler(sentence, swords, legals)
    for word in wrangled[0]:
        if word in bag_of_words:
            if word not in all_huck_words:
                all_huck_words[word] = 1
            else:
                all_huck_words[word] += 1
        else: 
            huck_left_out.append(word)

In [342]:
len(all_huck_words)  # we expect this to be 24944, the len of bag_of_words

24944

In [331]:
sorted(all_huck_words.items())[:10]

[(u'aaem', 0),
 (u'ab', 3),
 (u'aback', 0),
 (u'abaft', 0),
 (u'abandon', 0),
 (u'abandoned', 0),
 (u'abandoning', 0),
 (u'abandonment', 0),
 (u'abaout', 0),
 (u'abased', 0)]

In [332]:
len(set(huck_left_out))  #number of unique words left out

1927

In [333]:
sorted(list(set(huck_left_out)), reverse=False)[:10]  #first 10

[u'abner',
 u'abolitionist',
 u'abram',
 u'abusing',
 u'accessed',
 u'ache',
 u'actin',
 u'actuly',
 u'additions',
 u'addled']

In [334]:
sorted(list(set(huck_left_out)), reverse=True)[:10]  #last 10

[u'zip',
 u'yuther',
 u'yow',
 u'yourn',
 u'yo',
 u'yit',
 u'yistiddy',
 u'yisterday',
 u'yirls',
 u'yers']

<h2>
A note about these left out words
</h2>
<div class=h1_cell>
<p>
I am keeping bag_of_words static for simplicity. But in reality, it is a growing thing. We should really add these left out words into bag_of_words and zero them out for eap, hpl and mws. As it is, we are kind of playing by gothic rules, ony using the words we saw in gothic authors. What would happen if we expanded bag_of_words to include all the new words we see in each new book? Would that move us closer or farther away from similarity with the gothic authors?
</div>

<h2>
Challenge 4
</h2>
<div class=h1_cell>
<p>
Build the huck_vector and compare with other 3. Remember to sort items so vectors align.
</div>

In [335]:
huck_sorted_items = []

for word in sorted(all_huck_words):
    temp = []
    temp.append(word)
    temp.append(all_huck_words[word])
    huck_sorted_items.append(tuple(temp)) 

In [336]:
huck_vector = [pair[1] for pair in huck_sorted_items]

In [337]:
huck_vector[:10]

[0, 3, 0, 0, 0, 0, 0, 0, 0, 0]

In [338]:
eap_huck = cosine_similarity(eap_vector, huck_vector)
eap_huck

0.5220519130559351

In [339]:
hpl_huck = cosine_similarity(hpl_vector, huck_vector)
hpl_huck

0.575496270052048

In [340]:
mws_huck = cosine_similarity(mws_vector, huck_vector)
mws_huck

0.48183022170429957


<div class=h1_cell>
<p>
Huck Finn is definitely less similar. Closest to Lovecraft. Hmmmmm. They were writing at roughly the same time period.
</div>

<h2>
Challenge 4
</h2>
<div class=h1_cell>
<p>
Let's try one my literary colleague tells me is the antithesis of gothic: Oliver Twist by Charles Dickens.
<p>
Same routine as Huckleberry Finn. Find it, bag it, vectorize it, cosine it with other 4.
</div>

In [343]:
import urllib2
url = "http://www.gutenberg.org/cache/epub/730/pg730.txt"
f = urllib2.urlopen(url)


alltext = ""

for line in f.readlines():
    alltext += line

allsentences = word_punct_tokenizer.tokenize(alltext.decode('utf-8'))

all_twist_words = bag_of_words.copy()
twist_left_out = []
for word in all_twist_words:
    all_twist_words[word] = 0
for sentence in allsentences:
    wrangled = sentence_wrangler(sentence, swords, legals)
    for word in wrangled[0]:
        if word in bag_of_words:
            if word not in all_twist_words:
                all_twist_words[word] = 1
            else:
                all_twist_words[word] += 1
        else: 
            twist_left_out.append(word)

In [344]:
len(all_twist_words)

24944

In [346]:
twist_sorted_items = []

for word in sorted(all_twist_words):
    temp = []
    temp.append(word)
    temp.append(all_twist_words[word])
    twist_sorted_items.append(tuple(temp)) 
twist_sorted_items[:10]

[(u'aaem', 0),
 (u'ab', 0),
 (u'aback', 0),
 (u'abaft', 0),
 (u'abandon', 1),
 (u'abandoned', 1),
 (u'abandoning', 0),
 (u'abandonment', 0),
 (u'abaout', 0),
 (u'abased', 0)]

In [347]:
len(set(twist_left_out))

2010

In [348]:
sorted(list(set(twist_left_out)), reverse=False)[:10]

[u'abase',
 u'ablutions',
 u'abound',
 u'absenting',
 u'abstractedly',
 u'abuts',
 u'acause',
 u'acceded',
 u'acceding',
 u'accessed']

In [349]:
sorted(list(set(twist_left_out)), reverse=True)[:10]

[u'zip',
 u'younker',
 u'yokel',
 u'yerself',
 u'xxxviii',
 u'xxxvii',
 u'xxxvi',
 u'xxxv',
 u'xxxix',
 u'xxxiv']

<h2>
Dang
</h2>
<div class=h1_cell>
<p>
Looks like sentence_wrangler is letting through preface page numbers, e.g., xxxv. If we know we are dealing with books, I suppose we could write a special sentence_wrangler that knows about the weird things we will see in books and throw them out. I kind of like the idea of having a library of sentence wranglers that are tuned to specific styles of text. Then you can choose the one that makes the most sense.
</div>

<h2>
Same left outs?
</h2>
<div class=h1_cell>
<p>
I wonder how much an overlap there is between words being left out of Huck Finn and words left out of Oliver Twist.
</div>

In [350]:
intersection = set(twist_left_out).difference(set(huck_left_out))
len(intersection)

1693

<div class=h1_cell>
<p>
A pretty big overlap.
</div>

<h2>
Ok, back to the problem
</h2>
<div class=h1_cell>
<p>
Build the twist_vector and compare with other 4.
</div>

In [351]:
twist_vector = [pair[1] for pair in twist_sorted_items]

In [352]:
eap_twist = cosine_similarity(eap_vector, twist_vector)
eap_twist

0.6556003623577531

In [33]:
hpl_twist = cosine_similarity(hpl_vector, twist_vector)
hpl_twist

0.5937931706997982

In [34]:
mws_twist = cosine_similarity(mws_vector, twist_vector)
mws_twist

0.5688023249478481

In [353]:
huck_twist = cosine_similarity(huck_vector, twist_vector)
huck_twist

0.5709846751186273

<h2>
Poe is winner this time
</h2>
<div class=h1_cell>
<p>
Wonder if Poe and Dickens knew each other. They were writing at roughly the same time. Maybe we are picking up on the language and jargon of the time?
</div>

<h2>
Challenge 5
</h2>
<div class=h1_cell>
<p>
I'm putting down a challenge. Find a book that has a cosine similarity value of below .5 for all 3 gothic authors. I was able to get below .4! You will get a shout-out if you can beat me.
<p>
To make exploration easier, I packaged up the code to produce the 3 values into a single function. For each book I explored, I saved it in my dropbox account and then got the url to it. That is what I passed into my function. You could do something similar with Google docs. Or change the url to a file path.
</div>

In [361]:
def check_book(url, bag, swords, legals):
    f = urllib2.urlopen(url)
    alltext = ""

    for line in f.readlines():
        alltext += line

    allsentences = word_punct_tokenizer.tokenize(alltext.decode('utf-8'))

    all_words = bag.copy()
    for word in all_words:
        all_words[word] = 0
    for sentence in allsentences:
        wrangled = sentence_wrangler(sentence, swords, legals)
        for word in wrangled[0]:
            if word in bag:
                if word not in all_words:
                    all_words[word] = 1
                else:
                    all_words[word] += 1
                
    sorted_items = []

    for word in sorted(all_words):
        temp = []
        temp.append(word)
        temp.append(all_words[word])
        sorted_items.append(tuple(temp)) 
        
    x_vector = [pair[1] for pair in sorted_items]
    
    eap_x = cosine_similarity(eap_vector, x_vector)
    hpl_x = cosine_similarity(hpl_vector, x_vector)
    mws_x = cosine_similarity(mws_vector, x_vector)
    
    return (eap_x, hpl_x, mws_x)

In [362]:
# test to make sure get same values as by hand above
check_book("http://www.gutenberg.org/cache/epub/730/pg730.txt", bag_of_words, swords, legals)  # twist

(0.6556003623577531, 0.594146408238699, 0.5691251809493426)

In [363]:
check_book("http://www.gutenberg.org/files/57141/57141-0.txt", bag_of_words, swords, legals)  # close

(0.24827214023748173, 0.24123167807551535, 0.2312175265968621)

In [364]:
check_book("http://www.gutenberg.org/files/57136/57136-0.txt", bag_of_words, swords, legals)  # my winner

(0.5598991946885765, 0.5489741103371224, 0.49926869027946075)

<h2>
Closing note
</h2>
<div class=h1_cell>
<p>
I'm still interested in using these word-frequency vectors to see what we can do. Next week we will take a look at another way to reason with words, i.e. word co-occurrence matrices.
</div>